In [1]:
import soundfile
import torch
from espnet_model_zoo.downloader import ModelDownloader
from espnet2.bin.asr_inference import Speech2Text

In [2]:
def get_tensor_info(tensor):
  info = []
  for name in ['requires_grad', 'is_leaf', 'retains_grad', 'grad_fn', 'grad']:
    info.append(f'{name}({getattr(tensor, name, None)})')
  return ' '.join(info)

In [3]:
# Set test wav for attention image extraction
TEST_DATA_PATH = "./../data/dev_clean"
WAV_LIST_PATH = TEST_DATA_PATH + "/wav.scp"
ANSWER_LIST_PATH = TEST_DATA_PATH + "/text"

file_name_list = []
speech_ans_list = []

with open(WAV_LIST_PATH, "r") as f:
    lines = f.readlines()
    for line in lines:
        num, name = line.split(' ')
        file_name_list.append(name[:-1])

with open(ANSWER_LIST_PATH, "r") as f:
    lines = f.readlines()
    for line in lines:
        speech_ans_list.append(line[17:])

# Prepare model
d = ModelDownloader()

speech2text = Speech2Text(
    **d.download_and_unpack('Shinji Watanabe/librispeech_asr_train_asr_transformer_e18_raw_bpe_sp_valid.acc.best'),
    # Decoding parameters are not included in the model file
    maxlenratio=0.0,
    minlenratio=0.0,
    beam_size=1,
    ctc_weight=1.0,
    lm_weight=0.0,
    penalty=0.0,
    nbest=1,
    out_mode="ctc"
)
# Add register hook for in encoder layers.
net = speech2text.asr_model
print(net)

ESPnetASRModel(
  (frontend): DefaultFrontend(
    (stft): Stft(n_fft=512, win_length=512, hop_length=128, center=True, normalized=False, onesided=True)
    (frontend): Frontend()
    (logmel): LogMel(sr=16000, n_fft=512, n_mels=80, fmin=0, fmax=8000.0, htk=False)
  )
  (specaug): SpecAug(
    (time_warp): TimeWarp(window=5, mode=bicubic)
    (freq_mask): MaskAlongAxis(mask_width_range=[0, 30], num_mask=2, axis=freq)
    (time_mask): MaskAlongAxis(mask_width_range=[0, 40], num_mask=2, axis=time)
  )
  (normalize): GlobalMVN(stats_file=/home/jmpark/.conda/envs/speech/lib/python3.8/site-packages/espnet_model_zoo/653d10049fdc264f694f57b49849343e/exp/asr_stats_raw_sp/train/feats_stats.npz, norm_means=True, norm_vars=True)
  (encoder): TransformerEncoder(
    (embed): Conv2dSubsampling6(
      (conv): Sequential(
        (0): Conv2d(1, 512, kernel_size=(3, 3), stride=(2, 2))
        (1): ReLU()
        (2): Conv2d(512, 512, kernel_size=(5, 5), stride=(3, 3))
        (3): ReLU()
      )
    

In [4]:
audio_num = 20 # selelct one of the wav in file_name_list
speech, rate = soundfile.read(file_name_list[audio_num])

In [5]:
class Hook():
    def __init__(self, module):
        self.hook_f = module.register_forward_hook(self.hook_f_fn)
        self.hook_b = module.register_backward_hook(self.hook_b_fn)
        self.target_output = None
        self.target_output_grad_in = []
        self.target_output_grad_out = []

    def hook_f_fn(self, module, input, output):
        self.target_output = output.clone()

    def hook_b_fn(self, module, grad_input, grad_output):
        self.target_output_grad_in.append(grad_input[0].clone())
        self.target_output_grad_out.append(grad_output[0].clone())

    def close(self):
        self.hook.remove()

In [6]:
def apply_hook(model, layer_idx, module_type, attn_type):
    # module_type : 'encoder', 'decoder'
    # attn_type : 'self_attn', 'src_attn'
    for name, module in model.named_modules():
        if f'{module_type}.{module_type}s.{layer_idx}.{attn_type}.linear_out' == name:
            hook = Hook(module=module)
    
    return hook

In [7]:
hook_0 = apply_hook(net, 0, 'encoder', 'self_attn')
hook_1 = apply_hook(net, 1, 'encoder', 'self_attn')

In [8]:
out, ctc_out = speech2text(speech)
ctc_argmax = ctc_out.argmax(2)

True


In [12]:
check_idx = 10
grad_temp = torch.ones_like(ctc_out)
print(grad_temp.shape)
grad_temp[0, check_idx, ctc_argmax[0,check_idx]] = 1

torch.Size([1, 95, 5000])


In [13]:
ctc_out.backward(grad_temp, retain_graph=True)

In [11]:
hook_0.target_output_grad_in[0]

tensor([[[ -0.6352,   5.3425,  11.7575,  ...,  -6.0813,  -9.3283,   5.3467],
         [ -0.7902,   3.2818,  18.4000,  ...,   1.0425, -12.9053,  -1.8569],
         [ -1.2621,   2.6474,  13.2524,  ...,  -9.4451,  -8.9629,  -6.7557],
         ...,
         [ -0.5952,  39.8598,  -7.9002,  ...,  41.3405,  62.3667,   2.8763],
         [  3.8325, -31.8117,   7.0444,  ..., -21.2777, -65.8514,  -0.8827],
         [  0.5570,  -1.9880,   4.6235,  ..., -18.7067, -21.4056,  -0.4950]]])